In [8]:
import kagglehub
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# kaggle urls / names from used datasets
urls = [
    "joebeachcapital/30000-spotify-songs",
    "tomigelo/spotify-audio-features",
    "imuhammad/audio-features-and-lyrics-of-spotify-songs",
    "maharshipandya/-spotify-tracks-dataset",
    "amitanshjoshi/spotify-1million-tracks",
    "mrmorj/dataset-of-songs-in-spotify",
    "thedevastator/spotify-tracks-genre-dataset",
]

dataframes = []

for idx, url in enumerate(urls):
    # download dataset from kaggle
    local_path = kagglehub.dataset_download(url)

    files = os.listdir(local_path)

    csv_files = [file for file in files if file.endswith('.csv')]
    
    # read all csv files downloaded from kaggle and collect them in one dataframe
    df = None
    for csv in csv_files:
        full_path = os.path.join(local_path, csv)
        
        df2 = pd.read_csv(full_path)
        if df is None:
            df = df2
        else:
            df = pd.concat([df, df2], axis=0)

    # store the concatenated dataframe
    dataframes.append(df)

/var/folders/x8/bv52m_c16jlg9mzb2wmzrjbh0000gn/T/ipykernel_3810/1461776945.py:32: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(full_path)


In [9]:
# column names of our target dataset
columns = ['id', 'name', 'artist', 'album', 'release_date', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence']

# the column names of the datasets in the same order as the target column names to allow easy mapping
dataset_column_names = [
    ['track_id', 'track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
    ['track_id', 'track_name', 'artist_name', None, None, 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
    ['track_id', 'track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
    ['track_id', 'track_name', 'artists', 'album_name', None, 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
    ['track_id', 'track_name', 'artist_name', None, 'year', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
    ['id', 'song_name', None, None, None, 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
    ['track_id', 'track_name', 'artists', 'album_name', None, 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence'],
]

# prioritize datasets with more columns available
order = [0, 2, 3, 6, 4, 1, 5]

In [10]:
# transform the dataset_column_names to a mapping dict that pandas can use to rename all columns
column_mappings = []

for column_names in dataset_column_names:
    mapping = {}

    for idx, column_name in enumerate(column_names):
        if column_name is None:
            continue

        mapping[column_name] = columns[idx]
    
    column_mappings.append(mapping)

In [11]:
# initialize an empty dataframe to collect all the data from the different sets
df_complete = pd.DataFrame(data=[], columns=columns)
id_cache = []

# collect the data from each dataset in the specified order
for idx in order:
    print(f'processing dataset {idx}')

    df = dataframes[idx]
    mapping = column_mappings[idx]
    
    # rename the columns to the shared target name
    df_temp = df.rename(columns=mapping)
    mapped_columns = list(mapping.values())
    
    # remove all additional columns that the dataset may contain
    df_subset = df_temp[mapped_columns]
    
    # remove all duplicates from the dataset
    df_subset = df_subset.drop_duplicates(subset=['id'])
    
    # remove all rows which were already provided by an earlier dataset
    df_cleaned = df_subset[~df_subset['id'].isin(id_cache)]

    # skip datasets with no new songs remaining
    if len(df_cleaned) == 0:
        print('no new song data')

    # concatenate the new dataset to the complete dataset
    id_cache.extend(df_cleaned['id'].tolist())
    if not df_complete.empty:
        df_complete = pd.concat([df_complete, df_cleaned], ignore_index=True)
    else:
        df_complete = df_cleaned
    
    print('appended song data')

processing dataset 0
appended song data
processing dataset 2
no new song data
appended song data
processing dataset 3
appended song data
processing dataset 6
no new song data
appended song data
processing dataset 4
appended song data
processing dataset 1
appended song data
processing dataset 5
appended song data


In [12]:
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence']
scaler = StandardScaler()
df_complete[features] = scaler.fit_transform(df_complete[features])

In [13]:
# store the complete dataset locally so that the data collection process only needs to run once
df_complete.to_csv(path_or_buf='data.csv')